# LSA

LSA (Latent Semantic Analysis) juga dikenal sebagai LSI (Latent Semantic Index). LSA menggunakan Bag of Word(BoW) model yang nantinya akan menghasilkan term-document matrix. Baris mewakili istilah dan kolom mewakili dokumen. LSA mempelajari topik laten dengan melakukan dekomposisi matriks pada matriks istilah dokumen menggunakan dekomposisi nilai Singular. Beberapa langkah yang harus dilakukan yaitu :

### 1. Crawling Data

Untuk dapat melakukan crawling data, gunakan library scrapy. Install terlebih dahulu scrapynya dengan mengetikkan perintah dibawah ini pada command prompt atau Anaconda Prompt.

In [ ]:
pip install scrapy

Lalu setelah proses selesai, jalankan perintah untuk melakukan setup pada Spider.

In [ ]:
scrapy startproject WebScrapingBerita

Selanjutnya untuk membuat scraper atau crawler, kita harus menambahkan file .py baru di dalam directory, buat file baru bernama berita.py lalu tuliskan script sebagai berikut 

In [ ]:
import scrapy


class BeritaSpider(scrapy.Spider):
    name = 'berita'
    allowed_domains = ['www.liputan6.com']
    start_urls = ['http://www.liputan6.com/']

    def parse(self, response):
            cont = response.css('.articles--iridescent-list--text-item__details')

            #collecting data
            kat = cont.css('.articles--iridescent-list--text-item__category')
            judul = cont.css('.articles--iridescent-list--text-item__title-link-text')
            c=0
            
            #combining the results
            for review in kat:
                yield{'Kategori': ''.join(review.xpath('.//text()').extract()),
                      'Judul': ''.join(judul[c].xpath('.//text()').extract()),
                
                    }
                c=c+1


Selanjutnya melalui terminal, masuk terlebih dahulu ke dalam directori WebScrapingBerita/WebScrapingBerita/spiders, lalu jalankan perintah:

In [ ]:
scrapy runspider berita.py -o Berita.csv

Maka akan tampil hasil dari crawling dalam bentuk file csv. Langkah selanjutnya yang perlu dilakukan adalah preprocessing.

### 2. Preprocessing

Data preprocessing adalah proses yang mengubah data mentah ke dalam bentuk yang lebih mudah dipahami. Proses ini penting dilakukan karena data mentah sering kali tidak memiliki format yang teratur. Pertama,lakukan import libary yang diperlukan dalam langkah preprocessing dan LSA.

In [2]:
# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
%matplotlib inline  
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

#import nltk
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize

#preprocessing
from nltk.corpus import stopwords  #stopwords
from nltk import word_tokenize,sent_tokenize # tokenizing
from nltk.stem import PorterStemmer,LancasterStemmer  # using the Porter Stemmer and Lancaster Stemmer and others
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer  # lammatizer from WordNet

# for named entity recognition (NER)
from nltk import ne_chunk

# vectorizers for creating the document-term-matrix (DTM)
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

#stop-words
stop_words=set(nltk.corpus.stopwords.words('indonesian'))

load dataset yang akan di preprocessing dengan menggunakan library pandas untuk membaca file csv.

In [3]:
df = pd.read_csv('berita.csv')
df.head() #menampilkan 5 data teratas

,Kategori,Judul
0,Infografis,Infografis Peta Jalur Mudik Lebaran 2022 Rawan...
1,Bola,Bola Ganjil: Kakak Beradik Penyumbang Gol Timnas
2,Infografis,"Infografis Kuota Haji 2022 Indonesia, Biaya pe..."
3,Surabaya,AHY Optimistis Emil Dardak Jadi Harapan Baru D...
4,Megapolitan,Tebet Eco Park Jakarta Selatan Resmi Dibuka un...


Proses selanjutnya hapus kolom 'Kategori' karena tidak terlalu berguna dalam proses LSA nantinya.

In [4]:
# drop the kategori data.
df.drop(['Kategori'],axis=1,inplace=True)
df.head(10)

,Judul
0,Infografis Peta Jalur Mudik Lebaran 2022 Rawan...
1,Bola Ganjil: Kakak Beradik Penyumbang Gol Timnas
2,"Infografis Kuota Haji 2022 Indonesia, Biaya pe..."
3,AHY Optimistis Emil Dardak Jadi Harapan Baru D...
4,Tebet Eco Park Jakarta Selatan Resmi Dibuka un...
5,Ketika Perajin Kain Daerah Berkolaborasi denga...
6,Sinopsis Film Ziarah yang Raih Best Screenplay...
7,Ramadhan di Yaman Berjalan Lancar Berkat Genca...
8,Kisah Antusiasme Pengusaha dan Pengemudi Bus J...
9,"Mengenal NRG Coin, Token Kripto Blockchain Energi"


Dilanjutkan dengan proses tokenizing untuk memotong kalimat menjadi kata dengan cara memotong kata pada white space atau spasi dan membuang karakter tanda baca. Di sini lakukan preprocessing data. 

In [5]:
#fungsi clean_text
def clean_text(headline):
  le=WordNetLemmatizer()
  word_tokens=word_tokenize(headline)
  tokens=[le.lemmatize(w) for w in word_tokens if w not in stop_words  and len(w)>3]
  cleaned_text=" ".join(tokens)
  return cleaned_text


In [6]:
# melakukan preprocessing pada data judul
df['judul_cleaned']=df['Judul'].apply(clean_text)
df.head(10)

,Judul,judul_cleaned
0,Infografis Peta Jalur Mudik Lebaran 2022 Rawan...,Infografis Peta Jalur Mudik Lebaran 2022 Rawan...
1,Bola Ganjil: Kakak Beradik Penyumbang Gol Timnas,Bola Ganjil Kakak Beradik Penyumbang Timnas
2,"Infografis Kuota Haji 2022 Indonesia, Biaya pe...",Infografis Kuota Haji 2022 Indonesia Biaya Jem...
3,AHY Optimistis Emil Dardak Jadi Harapan Baru D...,Optimistis Emil Dardak Jadi Harapan Baru Demok...
4,Tebet Eco Park Jakarta Selatan Resmi Dibuka un...,Tebet Park Jakarta Selatan Resmi Dibuka Umum
5,Ketika Perajin Kain Daerah Berkolaborasi denga...,Ketika Perajin Kain Daerah Berkolaborasi Desai...
6,Sinopsis Film Ziarah yang Raih Best Screenplay...,Sinopsis Film Ziarah Raih Best Screenplay ASEA...
7,Ramadhan di Yaman Berjalan Lancar Berkat Genca...,Ramadhan Yaman Berjalan Lancar Berkat Gencatan...
8,Kisah Antusiasme Pengusaha dan Pengemudi Bus J...,Kisah Antusiasme Pengusaha Pengemudi Jelang Mu...
9,"Mengenal NRG Coin, Token Kripto Blockchain Energi",Mengenal Coin Token Kripto Blockchain Energi


Lalu hilangkan kolom 'Judul'

In [7]:
df.drop(['Judul'],axis=1,inplace=True)

In [8]:
df.head(10)

,judul_cleaned
0,Infografis Peta Jalur Mudik Lebaran 2022 Rawan...
1,Bola Ganjil Kakak Beradik Penyumbang Timnas
2,Infografis Kuota Haji 2022 Indonesia Biaya Jem...
3,Optimistis Emil Dardak Jadi Harapan Baru Demok...
4,Tebet Park Jakarta Selatan Resmi Dibuka Umum
5,Ketika Perajin Kain Daerah Berkolaborasi Desai...
6,Sinopsis Film Ziarah Raih Best Screenplay ASEA...
7,Ramadhan Yaman Berjalan Lancar Berkat Gencatan...
8,Kisah Antusiasme Pengusaha Pengemudi Jelang Mu...
9,Mengenal Coin Token Kripto Blockchain Energi


### 3. Ekstraksi Fitur dan Membuat Document-Term-Matrix(DTM)

Dalam DTM nilainya adalah nilai TFidf.

Saya juga telah menentukan beberapa parameter dari vectorizer Tfidf.

Beberapa poin penting:-

**1) LSA umumnya diimplementasikan dengan nilai Tfidf di mana-mana dan bukan dengan Count Vectorizer.**

**2) max_features bergantung pada daya komputasi Anda dan juga pada eval. metrik (skor koherensi adalah metrik untuk model topik). Coba nilai yang memberikan evaluasi terbaik. metrik dan tidak membatasi kekuatan pemrosesan.**

**3) Nilai default untuk min_df & max_df bekerja dengan baik.**

**4) Dapat mencoba nilai yang berbeda untuk ngram_range.**

In [12]:
vect =TfidfVectorizer(stop_words=stop_words,max_features=1000) # to play with. min_df,max_df,max_features etc...

In [13]:
vect_text=vect.fit_transform(df['judul_cleaned'])

Sekarang kita dapat melihat kata-kata yang paling sering dan langka di headline berita berdasarkan skor idf. Semakin kecil nilainya.

In [14]:
print(vect_text.shape)
print(vect_text)

(52, 319)
  (0, 41)	0.3963441324825203
  (0, 259)	0.3963441324825203
  (0, 5)	0.2315858531330811
  (0, 172)	0.2633364363176198
  (0, 212)	0.2386486585421916
  (0, 126)	0.3963441324825203
  (0, 244)	0.4378516258949888
  (0, 115)	0.3963441324825203
  (1, 300)	0.375237916193398
  (1, 235)	0.4145350422463308
  (1, 42)	0.4145350422463308
  (1, 137)	0.4145350422463308
  (1, 97)	0.4145350422463308
  (1, 53)	0.4145350422463308
  (2, 288)	0.35407116943965256
  (2, 132)	0.39115158902605723
  (2, 48)	0.39115158902605723
  (2, 114)	0.3073553655740414
  (2, 103)	0.39115158902605723
  (2, 164)	0.39115158902605723
  (2, 5)	0.20688555001662842
  (2, 115)	0.35407116943965256
  (3, 129)	0.4145350422463308
  (3, 73)	0.4145350422463308
  (3, 105)	0.4145350422463308
  :	:
  (48, 288)	0.3237121609307931
  (49, 240)	0.4420944202490175
  (49, 154)	0.4420944202490175
  (49, 76)	0.4420944202490175
  (49, 314)	0.4420944202490175
  (49, 239)	0.4001847180784101
  (49, 212)	0.2409611706380178
  (50, 289)	0.36181616

In [15]:
idf=vect.idf_

In [16]:
dd=dict(zip(vect.get_feature_names(), idf))
l=sorted(dd, key=(dd).get)
# print(l)
print(dd['covid'])
print(dd['sinopsis'])

3.3608540011180215
4.277144732992177


Dapat terlihat bahwa berdasarkan **nilai idf** , **'covid'** adalah kata **paling sering** sedangkan **'sinopsis'** **paling jarang** muncul di antara berita.

### LSA (Latent Semantic Analysis)

**LSA pada dasarnya adalah dekomposisi nilai tunggal.**

**SVD menguraikan DTM asli menjadi tiga matriks S=U.(sigma).(V.T). Di sini matriks U menunjukkan matriks topik-dokumen sedangkan (V) adalah matriks istilah-topik.**

**Setiap baris dari matriks U(matriks term dokumen) adalah representasi vektor dari dokumen yang bersangkutan. Panjang vektor ini adalah jumlah topik yang diinginkan. Representasi vektor untuk suku-suku dalam data kami dapat ditemukan dalam matriks V (matriks istilah-topik).**

Jadi, SVD memberi kita vektor untuk setiap dokumen dan istilah dalam data kita. Panjang setiap vektor adalah k. **Kemudian kita dapat menggunakan vektor-vektor ini untuk menemukan kata dan dokumen serupa menggunakan metode kesamaan kosinus.**

Kita dapat menggunakan fungsi truncatedSVD untuk mengimplementasikan LSA. Parameter n_components adalah jumlah topik yang ingin kita ekstrak.
Model tersebut kemudian di fit dan ditransformasikan pada hasil yang diberikan oleh vectorizer.

**Terakhir perhatikan bahwa LSA dan LSI (I untuk pengindeksan) adalah sama dan yang terakhir hanya terkadang digunakan dalam konteks pencarian informasi.**

In [17]:
from sklearn.decomposition import TruncatedSVD
lsa_model = TruncatedSVD(n_components=10, algorithm='randomized', n_iter=10, random_state=42)

lsa_top=lsa_model.fit_transform(vect_text)


In [18]:
print(lsa_top)
print(lsa_top.shape)  # (no_of_doc*no_of_topics)

[[ 4.79906349e-01 -2.55465246e-01 -3.54244632e-02 -2.41663524e-01
   3.81980295e-02  1.37193408e-01 -1.08157366e-01  2.44123479e-02
   2.58942410e-02 -2.12210430e-01]
 [ 5.54831363e-06 -2.26069877e-05 -1.44172400e-04 -9.74354526e-04
   5.95648969e-03  1.80944777e-02  2.13900808e-02  5.78919511e-02
   5.72398259e-01  1.51078263e-01]
 [ 1.75219482e-01 -1.12552307e-01 -2.76982311e-02 -3.09172781e-01
   2.89581388e-01 -1.57181147e-01  1.95679764e-01  1.20434615e-01
   5.60747202e-02 -1.40853507e-01]
 [ 3.98118319e-03 -7.77138367e-03 -3.17039265e-03 -3.22414729e-02
   2.55098825e-01 -1.74386187e-01  1.72819030e-01 -1.10409951e-01
  -6.85012773e-02  4.86634441e-02]
 [-4.61511199e-06 -1.05583869e-04 -9.65736932e-05  2.77536657e-04
   5.69450352e-03 -1.98087414e-03 -3.04442240e-02  3.00665439e-01
  -1.46507276e-01  3.39223543e-02]
 [ 9.72589047e-02 -3.78027667e-02 -1.22718828e-02 -1.63026272e-01
  -1.02165987e-01 -9.89364733e-02  6.65692227e-02  3.43758161e-02
   3.60189562e-02  3.86650953e-02

In [19]:
l=lsa_top[0]
print("Document 0 :")
for i,topic in enumerate(l):
  print("Topic ",i," : ",topic*100)
  


Document 0 :
Topic  0  :  47.9906348536701
Topic  1  :  -25.546524611097755
Topic  2  :  -3.5424463177622427
Topic  3  :  -24.16635241641774
Topic  4  :  3.8198029509193794
Topic  5  :  13.719340801974521
Topic  6  :  -10.81573655636145
Topic  7  :  2.441234790310862
Topic  8  :  2.589424104464471
Topic  9  :  -21.22104295696171


Mirip dengan dokumen lain kita bisa melakukan ini. Namun perhatikan bahwa nilai tidak menambah 1 seperti di LSA itu bukan kemungkinan topik dalam dokumen.

In [20]:
print(lsa_model.components_.shape) # (no_of_topics*no_of_words)
print(lsa_model.components_)

(10, 319)
[[ 1.62358412e-02  1.62358412e-02  3.62025215e-02 ...  5.91705159e-02
  -2.22056348e-06 -6.92364328e-07]
 [ 5.47874533e-02  5.47874533e-02 -2.72328836e-02 ... -4.41224731e-02
   4.20910224e-05 -5.14194315e-05]
 [ 1.82915134e-03  1.82915134e-03 -2.97615910e-03 ...  3.14387821e-02
  -1.84628694e-04 -9.04083372e-05]
 ...
 [-5.02357493e-03 -5.02357493e-03 -4.20192571e-02 ... -2.91797024e-02
  -3.46058222e-02  1.28159171e-03]
 [ 5.02322506e-03  5.02322506e-03 -2.01890900e-02 ...  1.17448495e-02
   3.47361406e-03 -1.16435366e-02]
 [-5.66125452e-02 -5.66125452e-02  1.55590759e-01 ...  5.63076176e-02
   5.44149302e-03 -2.26089877e-04]]


Sekarang bisa mendapatkan daftar kata-kata penting untuk masing-masing dari 10 topik seperti yang ditunjukkan.

In [22]:
# most important words for each topic
vocab = vect.get_feature_names()

for i, comp in enumerate(lsa_model.components_):
    vocab_comp = zip(vocab, comp)
    sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:10]
    print("Topic "+str(i)+": ")
    for t in sorted_words:
        print(t[0],end=" ")
    print("\n")
         

Topic 0: 
mudik lebaran 2022 juta jelang vaksin booster warga rawan menko 

Topic 1: 
19 covid vaksin pemudik suntik booster dadakan april positif imbau 

Topic 2: 
milan liga inter italia roma link live streaming jalan dapatkan 

Topic 3: 
juta warga jelang booster vaksin bekasi antusias malang perjalanan mudik 

Topic 4: 
indonesia menko mitratel optimistis menara pengembangan telekomunikasi pemudik endemi harap 

Topic 5: 
kripto token energi menko blockchain coin mengenal jalur pemudik kosong 

Topic 6: 
kripto token energi blockchain coin mengenal mitratel optimistis indonesia anak 

Topic 7: 
fakta pedagang muncul tebet gerd hoaks kabar kematian mendadak menyebabkan 

Topic 8: 
timnas hasil 23 garuda muda pohang steelers takluk beradik bola 

Topic 9: 
jasa marga mobil hati pemudik 10 catat jabotabek ribu tinggalkan 

